In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy as sp
import scipy.stats as scst
import matplotlib.pyplot as plt

In [2]:
b_growth = pd.read_csv('growth-raw.csv')
plate_map = pd.read_csv('platemap.csv')

In [3]:
b_growth.shape

(4421, 98)

In [4]:
#Each row of the example file contains the :
#Time (in seconds), 
#Temperature (in degrees Celsius), 
#absorbance readings at 600 nm 
#for the 96 wells in a microtiter plate over a 24-hour period. 

In [5]:
b_growth.head()

,Time,Temperature,A1,A2,A3,A4,A5,A6,A7,A8,...,H3,H4,H5,H6,H7,H8,H9,H10,H11,H12
0,0,28.2,0.0777,0.0811,0.0801,0.0800,0.0799,0.0806,0.0811,0.0800,...,0.0894,0.0889,0.0903,0.0894,0.0890,0.0896,0.0894,0.0923,0.0894,0.0908
1,20,28.9,0.0778,0.0811,0.0800,0.0801,0.0800,0.0817,0.0813,0.0800,...,0.0894,0.0889,0.0902,0.0904,0.0889,0.0896,0.0888,0.0922,0.0894,0.0909
2,40,29.3,0.0779,0.0815,0.0802,0.0802,0.0804,0.0816,0.0813,0.0801,...,0.0894,0.0888,0.0899,0.0903,0.0889,0.0895,0.0888,0.0932,0.0894,0.0909
3,60,29.8,0.0780,0.0814,0.0800,0.0801,0.0800,0.0816,0.0810,0.0802,...,0.0893,0.0888,0.0900,0.0903,0.0888,0.0895,0.0888,0.0931,0.0893,0.0906
4,80,30.2,0.0779,0.0813,0.0802,0.0801,0.0800,0.0816,0.0812,0.0801,...,0.0893,0.0888,0.0901,0.0903,0.0888,0.0895,0.0887,0.0931,0.0893,0.0906


## Tidy - make long format

In [6]:
m_growth = b_growth.melt(id_vars=['Time','Temperature'],
                             value_vars=b_growth.columns[2:],
                             value_name='OD600',
                             var_name='Well')

In [7]:
m_growth.head()

,Time,Temperature,Well,OD600
0,0,28.2,A1,0.0777
1,20,28.9,A1,0.0778
2,40,29.3,A1,0.0779
3,60,29.8,A1,0.0780
4,80,30.2,A1,0.0779


## Annotate data with strains and conditions

In [8]:
plate_map.head()

,Well,Strain,Environment
0,B2,A,1
1,B3,B,1
2,B4,C,1
3,B5,NaN,1
4,B6,A,2


In [9]:
map_growth = pd.merge(m_growth,plate_map,on='Well',how='inner')

In [10]:
map_growth.dtypes

Time             int64
Temperature    float64
Well            object
OD600          float64
Strain          object
Environment      int64
dtype: object

## Save anmotated file

In [11]:
map_growth.to_csv("annotated_growth.csv",sep=',')

## summarise data

In [12]:
gr_growth = map_growth.drop('Well',axis=1).groupby(['Strain','Environment','Time'])['OD600'].agg(['count',np.mean,np.std])

In [13]:
gr_growth.head()

count      mean       std
Strain Environment Time                           
A      1           0         8  0.927612  0.714551
                   20        8  0.921225  0.716627
                   40        8  0.875088  0.756127
                   60        8  0.872537  0.756947
                   80        8  0.872800  0.757253

In [14]:
ci_growth=gr_growth.assign(CI_L=lambda x: sp.stats.t.interval(0.95, x['count']-1, loc=x['mean'], scale=x['std']/np.sqrt(x['count']))[0],
                CI_R=lambda x: sp.stats.t.interval(0.95, x['count']-1, loc=x['mean'], scale=x['std']/np.sqrt(x['count']))[1])

In [15]:
growth_plot_df=ci_growth.reset_index()

In [24]:
plot_df=ci_growth.reset_index()
map_growth.plot.hist()

In [ ]:
map_growth

In [25]:
sns.set(style="darkgrid")
g = sns.FacetGrid(map_growth.sample(100), row="Environment",col='Strain', margin_titles=True)
g.map(plt.plot, 'Time','OD600')

In [23]:
tips = sns.load_dataset("tips")
g = sns.FacetGrid(tips, row="sex", col="time", margin_titles=True)
bins = np.linspace(0, 60, 13)
g.map(plt.hist, "total_bill", color="steelblue", bins=bins, lw=0)